In [ ]:
import threading
from queue import Queue
import random
import time

## Locks

In [ ]:
lock = threading.Lock()
z = 0

In [ ]:
def f(text, i):
    global z
    print(f"Sleeping... {text} ")
    time.sleep(5-i)
    print(f"AWAKE: {text}!")
    with lock:
        z+=1
        time.sleep(5-i)
        print(z, f"was added by {text}")

In [ ]:
threads = []
for i in range(5):
    t = threading.Thread(target=f, args=(f"x{i}", i))
    threads.append(t)
    t.start()
# [t.join() for t in]

## Semaphores

In [ ]:
semaphore = threading.Semaphore(2)
z = 0

In [ ]:
def s(text, i):
    global z
    with semaphore:
        print(f"Welcome... {text} ")
        print(f"{text} will sleep for {5-i}")
        z+=1
        time.sleep(5-i)
        print(z, f"was added by {text}")

In [ ]:
threads = []
for i in range(5):
    t = threading.Thread(target=s, args=(f"x{i}", i))
    threads.append(t)
    t.start()
[t.join() for t in threads]

## Queues

In [ ]:
def create_item(i):
    sleep_time = random.random()
    print(f"creating {i}")
    time.sleep(sleep_time)
    return i

def use_item(i):
    sleep_time = random.random()
    time.sleep(sleep_time)
    print(f"used {i}")

In [ ]:
POISON_PILL = "MUAHHAHAHAHAHAH"
def producer(queue):
    for i in range(10):
        item = create_item(i)
        queue.put(item)
    queue.put(POISON_PILL)
        
def consumer(queue):
    while True:
        print("Waiting...")
        item = queue.get()
        
        if item is not POISON_PILL:        
            use_item(item)
            queue.task_done()
            print(f"Done {item}")
        else:
            print("POISON_PILL'D... finito")
            queue.task_done()
            break

In [ ]:
queue = Queue()
t1 = threading.Thread(target=producer, args=(queue,) )
t2 = threading.Thread(target=consumer, args=(queue,) )

t1.start()
t2.start()
t1.join()
t2.join()
queue.join()

# Executor

In [ ]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor


In [ ]:
def return_it(item, timeout):
    time.sleep(timeout)
    return item

In [ ]:

with ProcessPoolExecutor(max_workers=None) as executor:
    future_1 = executor.submit(return_it, "item_5", 5)
    future_2 = executor.submit(return_it, "item_4", 4)
    future_3 = executor.submit(return_it, "item_3", 3)

print(future_1.result(), future_2.result(), future_3.result())

# Takes 5 seconds

In [ ]:
executor = ThreadPoolExecutor()
arg_combos = [("item_5", 5), ("item_4", 4), ("item_3", 3)]

In [ ]:
future = executor.submit(return_it, "item_5", 2)
future.add_done_callback(lambda x: print(f"Im done {x.result()}!"))

In [ ]:
executor.shutdown()

In [ ]:
future.result()

In [ ]:
executor.shutdown()

# MultiProcess

# AsyncIO

## Py3.5

In [14]:
import asyncio


async def delayed_hello(text, sleep_time):
    print(f"Started {text}")
    await asyncio.sleep(sleep_time)
    print(f"Finished {text}")
    return text


async def coro_increment(id):
    i = 0
    for i in range(1, 6):
        print(f"{id} has run for {i} seconds")
        await asyncio.sleep(1)
        print(f"I ({id}) got access back")
    return i

loop = asyncio.get_event_loop()

future1 = asyncio.ensure_future(delayed_hello("v1", 3))
future2 = asyncio.ensure_future(coro_increment("v2"))
future3 = asyncio.ensure_future(coro_increment("v3"))

loop.run_until_complete(future1)  # doesn't complete future2 and future3

gathered = asyncio.gather(future1, future2, future3)

# True, False, False
print("Completion Status:".center(30,"="), future1.done(), future2.done(), future3.done())
   
loop.run_until_complete(gathered)  # waits for future2 and future3 to finish

# True, True, True
print("Completion Status:".center(30,"="), future1.done(), future2.done(), future3.done())


RuntimeError: This event loop is already running

Started v1
v2 has run for 1 seconds
v3 has run for 1 seconds
I (v2) got access back
v2 has run for 2 seconds
I (v3) got access back
v3 has run for 2 seconds
I (v2) got access back
v2 has run for 3 seconds
I (v3) got access back
v3 has run for 3 seconds
Finished v1
I (v2) got access back
v2 has run for 4 seconds
I (v3) got access back
v3 has run for 4 seconds
I (v2) got access back
v2 has run for 5 seconds
I (v3) got access back
v3 has run for 5 seconds
I (v2) got access back
I (v3) got access back


## Py3.7

In [15]:
import asyncio


async def long_IO():
    print("[Long] start")
    await asyncio.sleep(2)
    print("[Long] Done")
    return "Done"


async def many_short_IO(id):
    print(f"[short] Lets go 1 - {id}")
    await asyncio.sleep(1)
    print(f"[short] carry on 2 - {id}")
    await asyncio.sleep(2)
    print(f"[short] almost there 3 - {id}")
    await asyncio.sleep(1)
    return f"[short] done! {id}"


async def main():

    future1 = asyncio.create_task(long_IO())
    future2 = asyncio.create_task(many_short_IO("v2"))
    future3 = asyncio.create_task(many_short_IO("v3"))

    await future1
    await future2, future3
    return [future1, future2, future3]

loop = asyncio.get_event_loop()
x = asyncio.run(main())


AttributeError: module 'asyncio' has no attribute 'run'